In [ ]:
#データベース作成、ベクトル埋め込み
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import os

# パラメータ
JSONL_FILE = "paragraphs.jsonl"
INDEX_FILE = "faiss_index.bin"
NPZ_FILE = "faiss_metadata.npz"
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
EMB_DIM = 384

# モデルロード
model = SentenceTransformer(MODEL_NAME)

# 既存のFAISS & npz 読み込み（なければ新規作成）
if os.path.exists(INDEX_FILE) and os.path.exists(NPZ_FILE):
    print("既存のFAISSインデックスとメタデータを読み込みます...")
    index = faiss.read_index(INDEX_FILE)
    npz_data = np.load(NPZ_FILE, allow_pickle=True)
    metadata_list = npz_data["metadata_list"].tolist()
    paragraphs = npz_data["paragraphs"].tolist()
    existing_ids = {m["id"] for m in metadata_list}
else:
    print("新規にインデックスとメタデータを作成します...")
    index = faiss.IndexFlatIP(EMB_DIM)
    metadata_list = []
    paragraphs = []
    existing_ids = set()

# JSONL から新規データだけ取得
new_paragraphs = []
new_metadata = []
new_ids = []

with open(JSONL_FILE, "r", encoding="utf-8") as f:
    for line in f:
        entry = json.loads(line)
        _id = entry["id"]
        if _id in existing_ids:
            continue  # すでに登録済み
        new_ids.append(_id)
        new_paragraphs.append(entry["paragraph"])
        new_metadata.append({
            "id": _id,
            "title": entry["title"],
            "page": entry["metadata"].get("page"),
            "page": entry["metadata"].get("author"),
            "source": entry["metadata"].get("source")
        })

print(f"新規に追加する段落数: {len(new_paragraphs)}")

if new_paragraphs:
    # ベクトル化
    embeddings = model.encode(new_paragraphs, batch_size=32, show_progress_bar=True)
    embeddings = np.array(embeddings).astype("float32")
    faiss.normalize_L2(embeddings)

    # FAISS に追加
    index.add(embeddings)

    # リスト更新
    paragraphs.extend(new_paragraphs)
    metadata_list.extend(new_metadata)

    # 保存
    faiss.write_index(index, INDEX_FILE)
    np.savez(NPZ_FILE, paragraphs=paragraphs, metadata_list=metadata_list)
    print("FAISS & .npz を差分更新しました。")
else:
    print("差分はありません。更新はスキップされました。")

